First version: September 22, 2016. Using Amtliches_Gemeindeverzeichnis.xls

Second version: October 26, 2016. Merge totals and per 100, add town ids, use the geojson file

In [1]:
import pandas as pd

In [31]:
df = pd.read_csv('data/final_dataset_totals.csv')
df_per100 = pd.read_csv('data/final_dataset_per100.csv')

## Delete duplicate columns

In [41]:
x = list(range(20, 91, 5))
y = list(range(24, 95, 5))
csvFemalecols = []
csvFemalecols_per100 = []
newMaleCols = []
newMaleCols_per100 = []
for a, b in zip(x, y):
    newMaleCols.append("{}-{}_men".format(a, b))
    newMaleCols_per100.append("{}-{}_men_per100".format(a, b))
    csvFemalecols.append("Ledige{}-{} JahreFrauen Total+/-".format(a, b))
    csvFemalecols_per100.append("{}-{}  jährigeFrauen +/- pro 100".format(a, b))

In [33]:
for col in csvFemalecols:
    del df[col]

In [34]:
# beware of the double whitespace
print(df_per100.columns[3])
print(csvFemalecols_per100[0])

25-29  jährigeFrauen +/- pro 100
20-24  jährigeFrauen +/- pro 100


In [35]:
for col in csvFemalecols_per100:
    del df_per100[col]

In [56]:
del df['Ledige Frauen Total +/-']


In [61]:
del df_per100[' Frauen  +/- pro 100']

## Change column names (no whitespaces)

In [74]:
df.columns = newColumns
df_per100.columns = newColumns_per100

In [75]:
df_per100.head()

,Gemeinde,singleMen_total_per100,20-24_men_per100,25-29_men_per100,30-34_men_per100,35-39_men_per100,40-44_men_per100,45-49_men_per100,50-54_men_per100,55-59_men_per100,60-64_men_per100,65-69_men_per100,70-74_men_per100,75-79_men_per100,80-84_men_per100,85-89_men_per100,90-94_men_per100
0,Aeugst am Albis,-1.4,0.6,0.4,0.2,0.1,-0.3,-0.2,-0.6,-0.9,-0.1,-0.1,-0.5,-0.3,-0.8,-0.8,-0.2
1,Affoltern am Albis,-0.9,0.7,0.6,0.6,0.4,0.3,-0.2,-0.2,-0.1,-0.2,-0.5,-0.5,-0.7,-0.7,-0.5,-0.3
2,Bonstetten,-1.9,0.3,-0.2,0.1,0.3,-0.4,0.2,-0.2,-0.2,-0.4,-0.2,-0.3,-0.5,-0.3,-0.2,-0.1
3,Hausen am Albis,-3.2,0.5,-0.4,0.0,0.0,-0.1,-0.4,0.1,-0.3,-0.8,-0.4,-0.2,-0.7,-0.4,-0.3,-0.1
4,Hedingen,0.3,0.4,0.3,0.9,0.5,0.1,-0.1,0.2,-0.4,-0.2,-0.3,-0.1,-0.1,-0.4,-0.3,-0.0


## Merge "total" and "per 100" datasets

In [79]:
df_total = df.merge(df_per100, how='left', on='Gemeinde')
df_total.to_csv('data/final_dataset_merged.csv', index=False)
df_total.head()

,Gemeinde,singleMen_total,20-24_men,25-29_men,30-34_men,35-39_men,40-44_men,45-49_men,50-54_men,55-59_men,...,45-49_men_per100,50-54_men_per100,55-59_men_per100,60-64_men_per100,65-69_men_per100,70-74_men_per100,75-79_men_per100,80-84_men_per100,85-89_men_per100,90-94_men_per100
0,Aeugst am Albis,-27,11,8,3,1,-5,-4,-12,-17,...,-0.2,-0.6,-0.9,-0.1,-0.1,-0.5,-0.3,-0.8,-0.8,-0.2
1,Affoltern am Albis,-111,82,71,67,47,37,-25,-20,-8,...,-0.2,-0.2,-0.1,-0.2,-0.5,-0.5,-0.7,-0.7,-0.5,-0.3
2,Bonstetten,-103,14,-13,6,18,-21,12,-12,-8,...,0.2,-0.2,-0.2,-0.4,-0.2,-0.3,-0.5,-0.3,-0.2,-0.1
3,Hausen am Albis,-113,19,-13,0,0,-2,-13,2,-11,...,-0.4,0.1,-0.3,-0.8,-0.4,-0.2,-0.7,-0.4,-0.3,-0.1
4,Hedingen,10,16,11,34,17,2,-5,7,-13,...,-0.1,0.2,-0.4,-0.2,-0.3,-0.1,-0.1,-0.4,-0.3,-0.0


## Add town id

In [208]:
towns_df = pd.read_excel('data/Amtliches_Gemeindeverzeichnis.xls', sheetname='Gemeindeliste-Liste d. communes', usecols=['GDENR','GDENAME'])

In [209]:
gname_xls = 'GDENAME'
gname_gpd = 'NAME'

df_test = df.merge(towns_df, how='outer', left_on='Gemeinde', right_on=gname_xls)
print("Merged DF shape (including rows which weren't merged)", df_test.shape)

print('Could not be merged part I - from the Town list')
df_test[df_test['Gemeinde'] != df_test['Gemeinde']][[gname_xls]]

Merged DF shape (including rows which weren't merged) (2330, 35)
Could not be merged part I - from the Town list


,GDENAME
2324,Belmont-Broye
2325,Gibloux
2326,Mont-Vully
2327,Surses
2328,Obersaxen Mundaun
2329,Jorat-Mézières


In [211]:
print('Could not be merged - part II: from the census data')
print(len(df_test[df_test['GDENAME'] != df_test['GDENAME']][['Gemeinde']]), 'rows found')
#df_test[df_test['GDENAME'] != df_test['GDENAME']][['Gemeinde']]

Could not be merged - part II: from the census data
43 rows found


In [95]:
new_towns = pd.read_excel('data/Amtliches_Gemeindeverzeichnis.xls', sheetname='neue Gemeinde - nlles communes', skiprows=18)
new_towns_2016 = new_towns[new_towns['Jahr\nAnnée\nAnno'] > 2015]

In [212]:
import re
def get_previous_towns(value):
    value = value.replace('Fusion de ', '')
    value = value.replace('Vereinigung von ', '')
    value = value.replace(' et ', ' und ')
    first_towns = value.split(', ')
    last_towns = first_towns[-1].split(' und ')
    previous_towns = first_towns[:-1]
    previous_towns.extend(last_towns)
    return previous_towns
get_previous_towns('Fusion de Bas-Vully et Haut-Vully')

['Bas-Vully', 'Haut-Vully']

In [114]:
new_towns_2016['previous_towns'] = new_towns_2016['Art der Entstehung\nMode de formation\nModo di formazione'].apply(get_previous_towns)

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [272]:
new_towns_2016.to_excel('data/new_towns.xls', index=False)

# Ok, let's merge this

In [263]:
df = pd.read_csv('data/final_dataset_merged.csv')
df.shape

(2324, 33)

In [264]:
df_merge = df.merge(towns_df, how='left', left_on='Gemeinde', right_on='GDENAME')
# df_merge = df.merge(towns_df, how='inner', left_on='Gemeinde', right_on='GDENAME') previous version
df_merge.shape

(2324, 35)

In [265]:
df_older_towns = df_merge[df_merge['GDENAME'] != df_merge['GDENAME']]
df_older_towns.shape

(43, 35)

### a) we "patch" df

In [266]:
import numpy as np

for index, row in new_towns_2016.iterrows():
    new_town = pd.DataFrame([{'Gemeinde': row['Neue Gemeinde\nNouvelle commune\nNuovo Comune'] }])
    previous_towns = row['previous_towns']
    singleMen_sums = []
    
    for previous_town in previous_towns:
        result = df_older_towns[df_older_towns['Gemeinde'] == previous_town]
        if len(result['Gemeinde']) < 1:
            print('Warning: no result for', previous_town)
        else:
            values = result.values.tolist()[0][1:-2] # numeric values only
            if len(singleMen_sums) == 0:
                singleMen_sums.extend(values)
            else:
                singleMen_sums = np.add(singleMen_sums, values)
    
    if len(singleMen_sums) > 0:
        new_row = [row['Neue Gemeinde\nNouvelle commune\nNuovo Comune']]
        new_row.extend(singleMen_sums)
        new_town = pd.DataFrame([new_row], columns = df.columns)
        df = df.append(new_town)

### b) we merge again

In [267]:
df_merge = df.merge(towns_df, how='inner', left_on='Gemeinde', right_on='GDENAME')
df_merge.shape

(2287, 35)

In [268]:
df[df.Gemeinde.str.contains('Surse')]

,Gemeinde,singleMen_total,20-24_men,25-29_men,30-34_men,35-39_men,40-44_men,45-49_men,50-54_men,55-59_men,...,45-49_men_per100,50-54_men_per100,55-59_men_per100,60-64_men_per100,65-69_men_per100,70-74_men_per100,75-79_men_per100,80-84_men_per100,85-89_men_per100,90-94_men_per100
574,Sursee,-216.0,15.0,-41.0,86.0,38.0,36.0,-8.0,0.0,-27.0,...,-0.1,0.0,-0.3,-0.3,-0.3,-0.3,-0.7,-0.9,-0.7,-0.3
0,Surses,-10.0,11.0,15.0,6.0,8.0,12.0,12.0,11.0,10.0,...,4.1,6.0,8.7,-0.1,-7.8,-2.7,-17.8,-14.0,-6.9,-5.5


In [270]:
del df_merge['GDENAME']

In [271]:
df_merge.to_csv('data/final_dataset_merged_id.csv', index=False, encoding='utf-8')

## Cast floats to int when not per 100

In [289]:
df = pd.read_csv('data/final_dataset_merged_id.csv')

In [290]:
from math import nan
def as_int(value):
    try:
        return int(value)
    except ValueError as e:
        print('Error with', e)
        return nan

In [291]:
col_list = df.columns[1:17]
for col in col_list:
    df[col] = df[col].apply(as_int)

In [292]:
df.to_csv('data/final_dataset_merged_id.csv', index=False)

# Add missing towns (older attempt)

In [16]:
df = pd.read_csv('data/final_dataset_totals_id.csv')

In [ ]:
import geopandas as gpd
towns_df = gpd.read_file('D3/gemeinden.json', encoding='utf-8')
del towns_df['EINWOHNERZ']
del towns_df['geometry']
del towns_df['id']

In [20]:
towns_df.head(1)

,BFS_NUMMER,NAME
0,3762,Scuol


In [216]:
df.head(1)

,Gemeinde,singleMen_total,20-24_men,25-29_men,30-34_men,35-39_men,40-44_men,45-49_men,50-54_men,55-59_men,...,45-49_men_per100,50-54_men_per100,55-59_men_per100,60-64_men_per100,65-69_men_per100,70-74_men_per100,75-79_men_per100,80-84_men_per100,85-89_men_per100,90-94_men_per100
0,Aeugst am Albis,-27,11,8,3,1,-5,-4,-12,-17,...,-0.2,-0.6,-0.9,-0.1,-0.1,-0.5,-0.3,-0.8,-0.8,-0.2


In [26]:
def remove_accents(value):
    return value.replace('é', '').replace('à', '').replace('ä', '').replace('ö','').replace('ü','').replace('â', '')

In [25]:
remove_accents('lac läman')

'lac lman'

In [22]:
df.merge(towns_df, how='right', left_on='Gemeinde', right_on='NAME')

,GDENR,Gemeinde,Ledige Männer Total +/-,Ledige Frauen Total +/-,Ledige20-24 JahreFrauen Total+/-,Ledige20-24 JahreMänner Total+/-,Ledige25-29 JahreFrauen Total+/-,Ledige25-29 JahreMänner Total+/-,Ledige30-34 JahreFrauen Total+/-,Ledige30-34 JahreMänner Total+/-,...,Ledige75-79 JahreFrauen Total+/-,Ledige75-79 JahreMänner Total+/-,Ledige80-84 JahreFrauen Total+/-,Ledige80-84 JahreMänner Total+/-,Ledige85-89 JahreFrauen Total+/-,Ledige85-89 JahreMänner Total+/-,Ledige90-94 JahreFrauen Total+/-,Ledige90-94 JahreMänner Total+/-,BFS_NUMMER,NAME
0,1.0,Aeugst am Albis,-27.0,27.0,-11.0,11.0,-8.0,8.0,-3.0,3.0,...,5.0,-5.0,16.0,-16.0,15.0,-15.0,4.0,-4.0,1,Aeugst am Albis
1,2.0,Affoltern am Albis,-111.0,111.0,-82.0,82.0,-71.0,71.0,-67.0,67.0,...,80.0,-80.0,86.0,-86.0,56.0,-56.0,30.0,-30.0,2,Affoltern am Albis
2,3.0,Bonstetten,-103.0,103.0,-14.0,14.0,13.0,-13.0,-6.0,6.0,...,25.0,-25.0,14.0,-14.0,8.0,-8.0,6.0,-6.0,3,Bonstetten
3,4.0,Hausen am Albis,-113.0,113.0,-19.0,19.0,13.0,-13.0,0.0,0.0,...,24.0,-24.0,14.0,-14.0,10.0,-10.0,4.0,-4.0,4,Hausen am Albis
4,5.0,Hedingen,10.0,-10.0,-16.0,16.0,-11.0,11.0,-34.0,34.0,...,4.0,-4.0,13.0,-13.0,12.0,-12.0,1.0,-1.0,5,Hedingen
5,6.0,Kappel am Albis,-9.0,9.0,-8.0,8.0,2.0,-2.0,-5.0,5.0,...,5.0,-5.0,6.0,-6.0,1.0,-1.0,1.0,-1.0,6,Kappel am Albis
6,7.0,Knonau,23.0,-23.0,-2.0,2.0,-20.0,20.0,-16.0,16.0,...,0.0,0.0,3.0,-3.0,6.0,-6.0,9.0,-9.0,7,Knonau
7,8.0,Maschwanden,18.0,-18.0,0.0,0.0,-4.0,4.0,-3.0,3.0,...,-1.0,1.0,8.0,-8.0,4.0,-4.0,-1.0,1.0,8,Maschwanden
8,9.0,Mettmenstetten,5.0,-5.0,-45.0,45.0,7.0,-7.0,-32.0,32.0,...,9.0,-9.0,10.0,-10.0,12.0,-12.0,10.0,-10.0,9,Mettmenstetten
9,10.0,Obfelden,-22.0,22.0,-9.0,9.0,-22.0,22.0,-11.0,11.0,...,18.0,-18.0,20.0,-20.0,20.0,-20.0,11.0,-11.0,10,Obfelden
